In [1]:
import geopandas as gpd
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.plot import plotting_extent
from shapely.geometry import Polygon, mapping
from rasterio.mask import mask
from os.path import join
import descartes
from PIL import Image

#import earthpy.spatial as es
#import earthpy.plot as ep
# import earthpy.clip as ec
# import earthpy.mask as em

import fiona 
import pyproj
import time

import warnings

In [2]:
# local paths
path_data = '/Volumes/other/datasets_and_ML/UNOSAT_Challenge/Train_Dataset'
area = 'Najaf_2015'


path_file = 's1tiling_S1A_IW_GRDH_1SDV_20150108T030926_20150108T030951_004074_004EC3_B908/38SLF/s1a_38SLF_vv_DES_20150108t030926.tif'
path_file_border_mask = 's1tiling_S1A_IW_GRDH_1SDV_20151222T030932_20151222T030957_009149_00D2A0_F04A/38SLF/s1a_38SLF_vv_DES_20151222t030932_BorderMask.tif'

path_test_raster_file = '/Volumes/other/datasets_and_ML/UNOSAT_Challenge/Train_Dataset/Nasiryah_2015/20150114_38RPV/s1a_38RPV_vh_ASC_20150114t145841.tif'
path_test_shape_file = '/Volumes/other/datasets_and_ML/UNOSAT_Challenge/Train_Dataset/Nasiryah_2015/38RPV_Nasiryah.shp'

shape_file = '38SLF_Mosul.shp'

In [47]:
def get_all_tiff_images_per_city(path_data, area):
    """get all .tif files per city which are no borderMask files
    """
    list_files_city = []
    for x in os.listdir(join(path_data, area)):
        if os.path.isdir(join(path_data, area, x)):
            dir_season = x
            for dir_lowest_level in os.listdir(join(path_data, area, dir_season)):
                y = join(path_data, area, dir_season, dir_lowest_level)
                if os.path.isdir(y):
                    for file_name in os.listdir(y):
                        if not 'BorderMask' in file_name and os.path.splitext(file_name)[1] == '.tif':
                            print(file_name)
                            list_files_city.append(join(path_data, area, dir_season, dir_lowest_level, file_name))

    return list_files_city

In [48]:
list_files_city = get_all_tiff_images_per_city(path_data, area)

s1a_38SLF_vh_DES_20150108t030926.tif
s1a_38SLF_vv_DES_20150108t030926.tif
s1a_38SLF_vh_20150426.tif
s1a_38SLF_vv_20150426.tif
s1a_38SLF_vh_DES_20150812t030933.tif
s1a_38SLF_vv_DES_20150812t030933.tif
s1a_38SLF_vh_DES_20151222t030932.tif
s1a_38SLF_vv_DES_20151222t030932.tif


In [49]:
list_raster_obs = []
for file in list_files_city:
    list_raster_obs.append(rio.open(file))

In [50]:
list_raster_obs

[<open DatasetReader name='/Volumes/other/datasets_and_ML/UNOSAT_Challenge/Train_Dataset/Mosul_2015/s1tiling_S1A_IW_GRDH_1SDV_20150108T030926_20150108T030951_004074_004EC3_B908/38SLF/s1a_38SLF_vh_DES_20150108t030926.tif' mode='r'>,
 <open DatasetReader name='/Volumes/other/datasets_and_ML/UNOSAT_Challenge/Train_Dataset/Mosul_2015/s1tiling_S1A_IW_GRDH_1SDV_20150108T030926_20150108T030951_004074_004EC3_B908/38SLF/s1a_38SLF_vv_DES_20150108t030926.tif' mode='r'>,
 <open DatasetReader name='/Volumes/other/datasets_and_ML/UNOSAT_Challenge/Train_Dataset/Mosul_2015/s1tiling_S1A_IW_GRDH_1SDV_20150426T030931_20150426T030956_005649_0073E6_2DFA/38SLF/s1a_38SLF_vh_20150426.tif' mode='r'>,
 <open DatasetReader name='/Volumes/other/datasets_and_ML/UNOSAT_Challenge/Train_Dataset/Mosul_2015/s1tiling_S1A_IW_GRDH_1SDV_20150426T030931_20150426T030956_005649_0073E6_2DFA/38SLF/s1a_38SLF_vv_20150426.tif' mode='r'>,
 <open DatasetReader name='/Volumes/other/datasets_and_ML/UNOSAT_Challenge/Train_Dataset/Mosul

In [54]:
for raster_obj in list_raster_obs:
    print(raster_obj) 
    print(raster_obj.bounds)
    print(raster_obj.meta)
    print()

<open DatasetReader name='/Volumes/other/datasets_and_ML/UNOSAT_Challenge/Train_Dataset/Mosul_2015/s1tiling_S1A_IW_GRDH_1SDV_20150108T030926_20150108T030951_004074_004EC3_B908/38SLF/s1a_38SLF_vh_DES_20150108t030926.tif' mode='r'>
BoundingBox(left=389200.0, bottom=4099890.0, right=391500.0, top=4099940.0)
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0.0, 'width': 230, 'height': 5, 'count': 1, 'crs': CRS.from_epsg(32638), 'transform': Affine(10.0, 0.0, 389200.0,
       0.0, -10.0, 4099940.0)}

<open DatasetReader name='/Volumes/other/datasets_and_ML/UNOSAT_Challenge/Train_Dataset/Mosul_2015/s1tiling_S1A_IW_GRDH_1SDV_20150108T030926_20150108T030951_004074_004EC3_B908/38SLF/s1a_38SLF_vv_DES_20150108t030926.tif' mode='r'>
BoundingBox(left=300000.0, bottom=3990240.0, right=409790.0, top=4100040.0)
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0.0, 'width': 10979, 'height': 10980, 'count': 1, 'crs': CRS.from_epsg(32638), 'transform': Affine(10.0, 0.0, 300000.0,
       0.0, -10.0, 410

In [ ]:
polygons = gpd.read_file(path_shape_file)
raster_obj= rio.open(path_raster)
